In [1]:
from decouple import config

In [2]:
CLIENT_ID = config("JD_APPKEY")
CLIENT_SECRET = config("JD_SECRET")
CLIENT_REDIRECT_URI = 'https://localhost'

In [3]:
from requests_oauthlib import OAuth2Session
import requests
import json

In [4]:
WELL_KNOWN_URL = 'https://signin.johndeere.com/oauth2/aus78tnlaysMraFhC1t7/.well-known/oauth-authorization-server'

In [5]:
wk_response = requests.get(WELL_KNOWN_URL)
wk_info = json.loads(wk_response.text)

AUTHORIZATION_URL = wk_info['authorization_endpoint']
TOKEN_GRANT_URL = wk_info['token_endpoint']
AVAILABLE_SCOPES = str(' ').join(wk_info['scopes_supported'])

print('Well Known Authorization URL - ' + AUTHORIZATION_URL)
print('Well Known Token Grant URL - ' + TOKEN_GRANT_URL)
print('Available Scopes - ' + AVAILABLE_SCOPES)

Well Known Authorization URL - https://signin.johndeere.com/oauth2/aus78tnlaysMraFhC1t7/v1/authorize
Well Known Token Grant URL - https://signin.johndeere.com/oauth2/aus78tnlaysMraFhC1t7/v1/token
Available Scopes - ag1 ag2 ag3 eq1 eq2 files jobs org1 org2 openid profile email address phone offline_access


In [6]:
REQUEST_SCOPES = {'ag1', 'files', 'org1'}
STATE = 'a1fo1'
oauth2_session = OAuth2Session(CLIENT_ID, redirect_uri=CLIENT_REDIRECT_URI, scope=REQUEST_SCOPES)

authorization_request, state = oauth2_session.authorization_url(AUTHORIZATION_URL, STATE)

print("Click on the following link to present the user with sign in form where they authenticate and approve access to your application.")
print(authorization_request) 

Click on the following link to present the user with sign in form where they authenticate and approve access to your application.
https://signin.johndeere.com/oauth2/aus78tnlaysMraFhC1t7/v1/authorize?response_type=code&client_id=johndeere-QUkAvtTF6g9ActPLwKsgXBbO5bXTqIr8AmoksW60&redirect_uri=https%3A%2F%2Flocalhost&scope=ag1+files+org1&state=a1fo1


In [9]:
AUTH_CODE = "METEMXSqC2gtuD6Iky85"

In [10]:
res_token = oauth2_session.fetch_token(TOKEN_GRANT_URL, code=AUTH_CODE, client_secret=CLIENT_SECRET)

KeyError: 'refresh_token'

In [13]:
access_token = res_token['access_token']
token_expires = res_token['expires_in']

In [14]:
MYJOHNDEERE_V3_JSON_HEADERS = { 'Accept': 'application/vnd.deere.axiom.v3+json',
                                'Content-Type': 'application/vnd.deere.axiom.v3+json'}

API_CATALOG_URL = "https://sandboxapi.deere.com/platform/"

api_catalog_res = oauth2_session.get(API_CATALOG_URL, headers=MYJOHNDEERE_V3_JSON_HEADERS)

In [15]:
api_catalog_res.status_code

200

In [20]:
org_url = None
for link in api_catalog_res.json()['links']:
    if link['rel'] == "organizations":
        org_url = link['uri']
        break

In [29]:
orgs = oauth2_session.get(org_url, headers=MYJOHNDEERE_V3_JSON_HEADERS)

In [49]:
orgs.json()['values'][0]['links']


[{'@type': 'Link',
  'rel': 'self',
  'uri': 'https://sandboxapi.deere.com/platform/organizations/468808'},
 {'@type': 'Link',
  'rel': 'machines',
  'uri': 'https://sandboxapi.deere.com/platform/organizations/468808/machines'},
 {'@type': 'Link',
  'rel': 'wdtCapableMachines',
  'uri': 'https://sandboxapi.deere.com/platform/organizations/468808/machines?capability=wdt'},
 {'@type': 'Link',
  'rel': 'files',
  'uri': 'https://sandboxapi.deere.com/platform/organizations/468808/files'},
 {'@type': 'Link',
  'rel': 'transferableFiles',
  'uri': 'https://sandboxapi.deere.com/platform/organizations/468808/files?transferable=true'},
 {'@type': 'Link',
  'rel': 'uploadFile',
  'uri': 'https://sandboxapi.deere.com/platform/organizations/468808/files'},
 {'@type': 'Link',
  'rel': 'sendFileToMachine',
  'uri': 'https://sandboxapi.deere.com/platform/organizations/468808/fileTransfers'},
 {'@type': 'Link',
  'rel': 'addMachine',
  'uri': 'https://sandboxapi.deere.com/platform/organizations/468808

In [50]:
boundary_url = None
for link in orgs.json()['values'][0]['links']:
    if link['rel'] == "boundaries":
        boundary_url = link['uri']
        break

In [51]:
boundary_url

'https://sandboxapi.deere.com/platform/organizations/468808/boundaries'

In [53]:
boundaries = oauth2_session.get(boundary_url, headers=MYJOHNDEERE_V3_JSON_HEADERS)

In [54]:
boundaries.json()

{'links': [{'rel': 'self',
   'uri': 'https://sandboxapi.deere.com/platform/organizations/468808/boundaries'},
  {'rel': 'nextPage',
   'uri': 'https://sandboxapi.deere.com/platform/organizations/468808/boundaries;start=10;count=10'}],
 'total': 45,
 'values': [{'@type': 'Boundary',
   'name': 'Boundary_3',
   'sourceType': 'DataCard',
   'createdTime': '2019-01-11T14:10:15.000Z',
   'modifiedTime': '2019-01-11T14:10:15.000Z',
   'area': {'@type': 'MeasurementAsDouble',
    'valueAsDouble': 12.987471668986617,
    'unit': 'ha'},
   'workableArea': {'@type': 'MeasurementAsDouble',
    'valueAsDouble': 12.323889733396133,
    'unit': 'ha'},
   'multipolygons': [{'@type': 'Polygon',
     'rings': [{'@type': 'Ring',
       'points': [{'@type': 'Point',
         'lat': 38.64895657644663,
         'lon': -78.69632125714834},
        {'@type': 'Point',
         'lat': 38.648956656910634,
         'lon': -78.6963516789243},
        {'@type': 'Point',
         'lat': 38.64892061064327,
        